# Window functions (OVER)

Check following sources:

- [SQL window functions](https://mode.com/sql-tutorial/sql-window-functions) tutorial on mode.com.
- [Window functions](https://www.postgresql.org/docs/9.1/tutorial-window.html) article on official postgres documentation.

A window function performs a calculation across a set of table rows that are somehow related to the current row. This is comparable to the type of calculation that can be done with an aggregate function. But unlike regular aggregate functions, use of a window function does not cause rows to become grouped into a single output row — the rows retain their separate identities. Behind the scenes, the window function is able to access more than just the current row of the query result.

In fact, you are doing something like aggregation, but the results are not collapsed by variable aggregation.

Let's consider an example for this page. We have a numeric variable called `some_variable`, on which we will apply various functions. Also, we have a `group` variable that represents the relationship between rows.

In [10]:
docker run -d --rm\
    --name window_functions_example\
    -e POSTGRES_PASSWORD=postgres\
    postgres:15.4 &> /dev/null
sleep 5

docker exec -i window_functions_example psql -U postgres -d postgres << EOF
CREATE TABLE example_table (
    some_value INT NOT NULL,
    "group" TEXT NOT NULL
);
INSERT INTO example_table (some_value, "group") VALUES
(2, 'C'),
(1, 'A'),
(3, 'A'),
(4, 'C'),
(4, 'A'),
(0, 'B'),
(0, 'A'),
(4, 'C'),
(3, 'B'),
(5, 'A');

SELECT * FROM example_table;
EOF

CREATE TABLE
INSERT 0 10
 some_value | group 
------------+-------
          2 | C
          1 | A
          3 | A
          4 | C
          4 | A
          0 | B
          0 | A
          4 | C
          3 | B
          5 | A
(10 rows)



**Note** Don't forget to stop the container.

In [9]:
docker stop window_functions_example

window_functions_example


## Syntax

If you are using a window function in the `SELECT` block of your query, you can declare a new output column using the following syntax:

```sql<function> OVER (<relationship>) AS <result name>.```

This allows you to compute the result of the window function and assign it to a new column with the specified `<result name>`.

Functions can be quite different, but actually there are only two components of `<relationship>`: `PARTITION` and `ORDER BY` , so consider them at the beginning.

## Partition

Allows you to define variables that divide rows into groups - rows in each group calculate it function values interpendently.

---

Consider the simplest possible example: for each record, calculate the sum of the `some_value` by the `group` of the row.

In [11]:
docker exec -i window_functions_example psql -U postgres -d postgres << EOF
SELECT 
    some_value, "group",
    SUM(some_value) OVER (PARTITION BY "group") AS result
FROM example_table;
EOF

 some_value | group | result 
------------+-------+--------
          0 | A     |     13
          1 | A     |     13
          3 | A     |     13
          5 | A     |     13
          4 | A     |     13
          0 | B     |      3
          3 | B     |      3
          4 | C     |     10
          4 | C     |     10
          2 | C     |     10
(10 rows)



Now, let's explore a more exotic example to better understand the flexibility of this tool. In this case, for each unique `some_value` we correspond emumeration of the possible values of `group` that ocur with that options of `some_value`.

In [12]:
docker exec -i window_functions_example psql -U postgres -d postgres << EOF
SELECT 
    some_value, "group",
    string_agg("group", ', ') OVER (PARTITION BY some_value) AS result
FROM example_table;
EOF

 some_value | group | result  
------------+-------+---------
          0 | B     | B, A
          0 | A     | B, A
          1 | A     | A
          2 | C     | C
          3 | A     | A, B
          3 | B     | A, B
          4 | A     | A, C, C
          4 | C     | A, C, C
          4 | C     | A, C, C
          5 | A     | A
(10 rows)



## Order by

The `ORDER BY` clause defines the order of application of the window function. Each unique value of the ordering variable in the aggregation will contain only values corresponding to the previous values according to the order. 

---

The following examples apply to the computation of sums of `some_value` in the order defined by `group`.

In [13]:
docker exec -i window_functions_example psql -U postgres -d postgres << EOF
SELECT 
    some_value, "group",
    SUM(some_value) OVER (ORDER BY "group") AS sv
FROM example_table;
EOF

 some_value | group | sv 
------------+-------+----
          0 | A     | 13
          1 | A     | 13
          3 | A     | 13
          5 | A     | 13
          4 | A     | 13
          0 | B     | 16
          3 | B     | 16
          4 | C     | 26
          4 | C     | 26
          2 | C     | 26
(10 rows)



So for `A` we only have the sum of the `some_values` of the rows corresponding to the `A` group. But for `B` we have the sum on both `A` and `B` groups and so on.